<a href="https://colab.research.google.com/github/vhmartinsp/nlp_assistente_virtual/blob/main/nlp_assistente_virtual_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformação texto para áudio

Importação de bibliotecas e módulos

In [1]:
!pip install gtts
!pip install SpeechRecognition


In [29]:
# Passo 1: Instalar o FFmpeg no Colab
!apt-get -y install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
import os
import speech_recognition as sr
from gtts import gTTS
import pyjokes
from datetime import datetime
from base64 import b64decode
from google.colab import output
from IPython.display import display, HTML
import webbrowser
from io import BytesIO
import subprocess
from IPython.display import Audio
import time
import wikipedia


Função em JavaScript para acessar o microfone via Colab

In [ ]:
# Função para capturar áudio do usuário
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
    print("Speak Now...")
    display(Javascript(RECORD))
    sec += 1  # Ajuste de tempo para garantir a captura adequada
    s = output.eval_js(f'record({sec * 1000})')
    print("Done Recording!")
    b = b64decode(s.split(',')[1])  # Decodificando o áudio
    return b  # Retorna como byte stream

In [26]:

# Função para converter o arquivo de áudio para WAV
def convert_to_wav(audio_data):
    input_file = "/content/temp_audio.mp3"
    output_file = "/content/temp_audio.wav"

    with open(input_file, "wb") as f:
        f.write(audio_data)

    # Usando FFmpeg para converter para WAV
    command = f"ffmpeg -i {input_file} {output_file}"
    subprocess.run(command, shell=True)

    with open(output_file, "rb") as f:
        wav_data = f.read()

    os.remove(input_file)
    os.remove(output_file)

    return wav_data

# Função para transcrever áudio usando a biblioteca speech_recognition
def transcribe_audio(audio_data):
    r = sr.Recognizer()
    with sr.AudioFile(BytesIO(audio_data)) as source:
        r.adjust_for_ambient_noise(source)  # Reduzir o ruído
        audio = r.record(source)
    try:
        text = r.recognize_google(audio, language='pt-BR')  # Usando a API do Google para português
        return text
    except sr.UnknownValueError:
        return "Desculpe, não entendi."
    except sr.RequestError:
        return "Desculpe, o serviço não está disponível."

def speak(text):
    tts = gTTS(text=text, lang='pt')
    filename = "voice.mp3"
    try:
        os.remove(filename)
    except OSError:
        pass
    tts.save(filename)

    # Usando IPython.display.Audio para reproduzir o áudio
    display(Audio(filename, autoplay=True))
    # Aguardar até que a reprodução do áudio termine
    time.sleep(4)  # Aumentado para dar mais tempo para a fala ser ouvida

def respond(text):
    print("Texto recebido: " + text)

    # Convertendo para minúsculas para garantir a correspondência com palavras-chave
    text = text.lower()

    if 'música do mozão' in text:
        speak("Vou colocar a música para você.")
        url = "https://www.youtube.com/watch?v=0Fs_mpX6aUY&list=PLEhSJuiHJf64_H8TN33TzYJ4RABWs7kJ9&index=10"
        # Substitui a abertura direta do navegador por um link clicável
        display(HTML(f'<a href="{url}" target="_blank">Clique aqui para ouvir a música do Mozão</a>'))
        speak("Aqui está a música para você no YouTube.")

    elif 'pesquisa' in text:
        speak("O que você quer pesquisar?")

        while True:
            query = get_audio()  # Obtém a consulta do usuário
            if query != '':
                try:
                    result = wikipedia.summary(query, sentences=3)  # Obtém um resumo da Wikipedia
                    speak(f"Aqui está o que encontrei sobre {query}.")
                    print(result)
                    speak(result)

                    speak("Você gostaria de fazer outra pesquisa? Diga 'sim' para continuar ou 'não' para sair.")
                    response = get_audio()
                    if 'não' in response.lower():
                        speak("Ok, até a próxima!")
                        break  # Sai do loop se o usuário não quiser mais pesquisar
                except wikipedia.exceptions.DisambiguationError as e:
                    speak(f"Houve uma ambiguidade na pesquisa. Existem várias opções para '{query}'. Você pode ser mais específico?")
                    print(f"Ambiguidade: {e.options}")
                except wikipedia.exceptions.HTTPTimeoutError:
                    speak("Houve um problema com a conexão à Wikipedia. Tente novamente mais tarde.")
                except wikipedia.exceptions.RequestError:
                    speak("Não consegui acessar a Wikipedia. Tente novamente mais tarde.")
                except Exception as e:
                    speak(f"Desculpe, ocorreu um erro: {e}")
                    break
            time.sleep(2)  # Atraso maior entre as pesquisas

    elif 'piada' in text:
        speak(pyjokes.get_joke())
    elif 'que horas' in text or 'que horas são?' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'sair' in text:
        speak("Até mais, até a próxima!")
        exit()
    else:
        speak("Desculpe, não entendi o comando.")

# Função para capturar áudio e tentar a transcrição
def get_audio():
    audio_data = record()  # Captura o áudio
    if audio_data:
        wav_data = convert_to_wav(audio_data)  # Converte para WAV
        text = transcribe_audio(wav_data)  # Transcreve o áudio para texto
        print(f"Texto transcrito: {text}")  # Verificando o texto transcrito
        return text
    return ''

# Função de controle de fluxo
def check_continue():
    speak("Você gostaria de continuar? Diga 'sim' para continuar ou 'não' para sair.")
    response = get_audio().lower()
    if 'não' in response:
        speak("Ok, até a próxima!")
        return False
    return True

# Loop principal de escuta e resposta
while True:
    print("Estou ouvindo...")
    text = get_audio()  # Captura o áudio
    if text.strip():  # Verifica se há texto transcrito
        respond(text)  # Responde ao comando
    if not check_continue():  # Pergunta se o usuário quer continuar
        break  # Sai do loop se o usuário não quiser continuar
    time.sleep(2)  # Aguarda 2 segundos antes de processar novamente


Estou ouvindo...
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Texto transcrito: pesquisa
Texto recebido: pesquisa


Speak Now...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Done Recording!
Texto transcrito: Desculpe, não entendi.


Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Texto transcrito: Desculpe, não entendi.
